In [74]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import csv

"Book of the Week" was implemented starting in season 7. Seasons 7-12 have their own pages on the sidebar, reachable at links like https://writingexcuses.com/category/season/season-07/

In [76]:
#base url
base = 'https://writingexcuses.com/category/season/season-'

#adding seasons
seasons = ['07/', '08/', '09/', '10/', '11/', '12/']

#append season numbers for ultimate list of desired URLs
urls = []
for season in seasons:
    urls.append(base + season)

In [7]:
#loop through list of URLs and parse for the audiobook image indicating a book of the week
l = []
for link in urls:
    url = link
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    l.append(soup.find_all("div", class_="wx_audiobook"))

In [44]:
# this flattens a nested list where the order (season in this case) doesn't matter
# not relevant to this code
k = [item for sublist in l for item in sublist]

In [16]:
#find the text under the 'a' tag in the first element of the soup
l[0][0].find('a').text

'Hard Magic'

In [34]:
#checking on the loop
j

<div class="wx_audiobook"><p>The Autumn Republic, by Brian McClellan</p>
</div>

In [35]:
#create empty list for book titles
books = []

#iterate through nested list
#append book title depending on the tag it falls under
for i in l:
    for j in i:
        if j.find('em') != None:
            books.append(j.find('em').text)
        elif j.find('a') != None:
            books.append(j.find('a').text)
        else:
            books.append(j.find('p').text)

In [77]:
#create empty list for all text associated with the audiobook div
alltext = []

#iterate through nested list
#append text for playing with later
for i in l:
    for j in i:
        alltext.append(j.find('p').text)

In [78]:
#create empty list for authors names to be pulled from alltext list
authors = []

#iterate through alltext list and use regular expressions to try to eliminate unneccesary text
for index, line in enumerate(alltext):
    line = line.replace(books[index], '')
    if re.search("by (.*), narrated by", line):
        authors.append(re.search("by (.*), narrated by", line).group(1))
    else:
        authors.append(line)

In [82]:
#check that lengths of lists are the same
len(authors) == len(books)

True

In [89]:
finaldf = pd.DataFrame(list(zip(books, authors)), columns =['Title', 'Author/Info'])
finaldf

,Title,Author/Info
0,"Hard Magic,",Larry Correia
1,"A Fire Upon the Deep,",Vernor Vinge
2,Speaker for the Dead,Orson Scott Card
3,"Farenheit 451,",Ray Bradbury
4,Terrorists in Love: The Real Stories of Islami...,Ken Ballen
5,Alloy of Law,"Our stuff! by Brandon Sanderson, Shades of Mi..."
6,"His Majesty’s Dragon: Temeraire, Book 1",Naomi Novik
7,"The Terror,",Dan Simmons
8,One Salt Sea,Seanan McGuire
9,"Startide Rising,",David Brin


In [90]:
#convert the two lists to a dictionary where the book titles are the keys and author text are the values
final = {books[i]: authors[i] for i in range(len(books))} 

In [91]:
#write to CSV
with open('BOWfromdict.csv', 'w', newline="") as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in final.items():
       writer.writerow([key, value])

In [92]:
#or, write to CSV from the dataframe
finaldf.to_csv('BOWfromcsv.csv')